In [1]:
import sounddevice as sd
from scipy.io.wavfile import write
import speech_recognition as sr
from gtts import gTTS
import time
import playsound
import librosa
import PySimpleGUI as sg
import numpy as np
import os
import librosa
import sys
from tensorflow.keras.models import load_model
import cv2 as cv

C:\Users\Utente\anaconda3\envs\Python3_6\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Utente\anaconda3\envs\Python3_6\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\Utente\anaconda3\envs\Python3_6\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
def crop(input, size):
    output = input[0:min(size, input.shape[0])]
    output = np.concatenate((output, np.zeros(size-output.shape[0])))
    return output


def mfcc(file_path, max_len=40):
    seconds=3
    wave, sr = librosa.load(file_path, mono=True, sr=44100)
    wave = crop(wave, sr*seconds)
    mfcc = librosa.feature.mfcc(wave, sr=sr, n_fft=2048, hop_length=512, n_mfcc=40)
    return mfcc


def main(input, model):
    
    dict_comm = {"data": 0, "ora": 1, "temperatura":2}
    
    #estraggo le features
    X = mfcc(input)
    X = X.reshape(-1,40,259,1)
    
    y_lau = np.argmax(model.predict(X)[0], axis=1)[0]
    y_g = np.argmax(model.predict(X)[1], axis=1)[0]
    y_d = np.argmax(model.predict(X)[2], axis=1)[0]

    y_az = np.argmax(model.predict(X)[3], axis=1)[0]
    azione = list(dict_comm.keys())[y_az]
    
    #subject = str()
    
    if y_lau==0 and y_g==0 and y_d==0:
        subject = 'Sconosciuto'
    elif y_lau==1 and y_g==0 and y_d==0:
        subject = 'Laura'
    elif y_lau==0 and y_g==1 and y_d==0:
        subject = 'Gabriele'
    elif y_lau==0 and y_g==0 and y_d==1:
        subject = 'Davide'
    else:
        subject = 'Sconosciuto'

    if azione=='data':
        azione='Data'
    elif azione=='ora':
        azione='Ora'
    else:
        azione='Temperatura'

    return subject, azione
    

def speak(text):
    tts = gTTS(text=text, lang='it')
    filename = 'C:\\Users\\Utente\\Desktop\\reply.mp3'
    tts.save(filename)
    playsound.playsound(filename)
    os.remove(filename)

In [3]:
PATH = 'C:\\Users\\Utente\\Desktop\\'

In [4]:
%time model = load_model(PATH+'model')

Wall time: 2.62 s


In [5]:
from datetime import datetime
import locale
locale.setlocale(locale.LC_ALL, 'it')

'it'

In [6]:
duration = 3
rec_rate = 44100

cap = cv.VideoCapture(0)
status=""
f=""

i=1

while(True):

    r, frame = cap.read()
    cv.putText(frame, f, (15, 37), cv.FONT_HERSHEY_SIMPLEX, 0.75, (255,255,255), 2)
    cv.imshow(' ', frame)
    
    # Registra premendo il tasto R
    if cv.waitKey(20) & 0xFF == ord('r'):
        f = 'Inizio a registrare...'
        audio = sd.rec(int(duration * rec_rate), samplerate=rec_rate, channels=1)
        sd.wait(3)

        write('temp.wav', rate=rec_rate, data=audio)

        subject_, azione_ = main('temp.wav', model)
        os.remove('temp.wav')
 
        if azione_== 'Ora':
            data= datetime.now()
            viz= str(int(data.strftime("%H"))) + ", " + str(int(data.strftime("%M")))+" minuti, e "+str(int(data.strftime("%S")))+ " secondi"
            if subject_ != 'Sconosciuto':
                output_text = "Ciao "+ subject_ + ", sono le ore "+ viz
            else:
                output_text = "Ciao, oggi e il "+ viz

        if azione_== 'Data':
            data= datetime.today()
            viz= str(int(data.strftime("%d")))+' '+str(data.strftime("%B"))+' '+str(int(data.strftime("%Y")))
            if subject_ != 'Sconosciuto':
                output_text = "Ciao "+ subject_ + ", oggi e il "+ viz
            else:
                output_text = "Ciao, oggi e il "+ viz
        
        #da gestire con API che permettano di ottenere informazioni meteo 
        if azione_== 'Temperatura':
            if subject_== 'Sconosciuto':
                output_text = "Ciao, la temperatura di oggi e "+ "5 gradi"
            elif subject_== 'Laura':
                output_text = "Ciao "+ subject_ + ", la temperatura di oggi e "+ "14 gradi"
            elif subject_== 'Davide':
                output_text = "Ciao "+ subject_ + ", la temperatura di oggi e "+ "14 gradi"
            elif subject_== 'Gabriele':
                output_text = "Ciao "+ subject_ + ", la temperatura di oggi e "+ "14 gradi"
            else:
                output_text = "Ciao, la temperatura di oggi e "+ "5 gradi"
            
        try:
            f = output_text
            speak(output_text)
        except:
            pass
        
    k = cv.waitKey(10) & 0xff
    if k == 27: # press 'ESC' to quit
        break
        
cap.release()
cv.destroyAllWindows()